In [1]:
import sqlite3

db_file="fund_database.db"


In [2]:
#connexion à la base de données
try:
    # Connexion à la base de données SQLite
    conn = sqlite3.connect(db_file)
    cursor = conn.cursor()

    # Création de la table Clients
    cursor.executescript("""
                CREATE TABLE IF NOT EXISTS Clients (
                    id INTEGER PRIMARY KEY AUTOINCREMENT, 
                    name TEXT NOT NULL UNIQUE, 
                    age INTEGER NOT NULL, 
                    country TEXT NOT NULL,
                    email TEXT NOT NULL UNIQUE,
                    risk_profile TEXT NOT NULL,
                    registration_date TEXT NOT NULL,
                    investment_amount REAL NOT NULL,
                    manager_id INTEGER NOT NULL,
                    portfolio_id INTEGER NOT NULL
                );

                CREATE TABLE IF NOT EXISTS Managers (
                    id INTEGER PRIMARY KEY AUTOINCREMENT, 
                    name TEXT NOT NULL UNIQUE, 
                    age INTEGER NOT NULL, 
                    country TEXT NOT NULL,
                    email TEXT UNIQUE NOT NULL,
                    seniority TEXT NOT NULL,
                    investment_sector TEXT NOT NULL
                );

                CREATE TABLE IF NOT EXISTS Manager_Strategies (
                    id INTEGER PRIMARY KEY AUTOINCREMENT,
                    manager_id INTEGER NOT NULL,
                    strategy TEXT NOT NULL,
                    FOREIGN KEY (manager_id) REFERENCES Managers(id),
                    UNIQUE (manager_id, strategy)  
                );

                CREATE TABLE IF NOT EXISTS Manager_Clients (
                    id INTEGER PRIMARY KEY AUTOINCREMENT,
                    manager_id INTEGER NOT NULL,
                    client_id INTEGER NOT NULL,
                    FOREIGN KEY (manager_id) REFERENCES Managers(id),
                    FOREIGN KEY (client_id) REFERENCES Clients(id),
                    UNIQUE (manager_id, client_id) 
                );

                CREATE TABLE IF NOT EXISTS Manager_Portfolios (
                    id INTEGER PRIMARY KEY AUTOINCREMENT,
                    manager_id INTEGER NOT NULL,
                    portfolio_id INTEGER NOT NULL,
                    FOREIGN KEY (manager_id) REFERENCES Managers(id),
                    FOREIGN KEY (portfolio_id) REFERENCES Portfolios(id),
                    UNIQUE (manager_id, portfolio_id) 
                );

                CREATE TABLE IF NOT EXISTS Portfolios (
                    id INTEGER PRIMARY KEY AUTOINCREMENT,
                    manager_id INTEGER NOT NULL,
                    client_id INTEGER NOT NULL,
                    strategy TEXT NOT NULL,
                    investment_sector TEXT NOT NULL,
                    size INTEGER NOT NULL,
                    value REAL NOT NULL,
                    FOREIGN KEY (client_id) REFERENCES Clients(id),
                    FOREIGN KEY (manager_id) REFERENCES Managers(id)
                );


                CREATE TABLE IF NOT EXISTS Portfolio_Products (
                    id INTEGER PRIMARY KEY AUTOINCREMENT,
                    portfolio_id INTEGER NOT NULL,
                    product_id INTEGER NOT NULL,
                    FOREIGN KEY (portfolio_id) REFERENCES Portfolios(id),
                    FOREIGN KEY (product_id) REFERENCES Products(id),
                    UNIQUE (portfolio_id, product_id) 
                );

                CREATE TABLE IF NOT EXISTS Products (
                    id INTEGER PRIMARY KEY AUTOINCREMENT,
                    ticker TEXT NOT NULL UNIQUE,
                    sector TEXT NOT NULL
                );
                         
        """)
    print("✅ Toutes les tables ont été créées avec succès.")


    conn.commit()
    
except sqlite3.Error as e:

    print(f"Erreur SQLite : {e}")

finally:
    # Fermeture de la connexion
    if conn:
        conn.close()

✅ Toutes les tables ont été créées avec succès.


In [3]:
#Nouveau client
from data_collector import generate_precise_client, generate_random_client, manager_affiliation, create_manager, create_portfolio

from base_builder import Client, AssetManager, Portfolio



try: 
    db = sqlite3.connect(db_file)
    
    sortie=False

    # 🎯 Demande à l'utilisateur son choix
    choice = input("Voulez-vous créer un client aléatoire (1) ou manuel (2) ? 📌 Entrez 1 ou 2 : ")

    if choice == "1":
        client_data = generate_random_client()
    elif choice == "2":
        client_data = generate_precise_client()
    else:
        print("❌ Choix invalide ! Par défaut, on crée un client aléatoire.")
        client_data = generate_random_client()

    print(client_data)

    assigned_manager = manager_affiliation(client_data, db)

    if assigned_manager:
        client_data["manager_id"] = assigned_manager["id"]
        print(f"✅ Manager attribué : {assigned_manager['name']}")


    else:
        print("⚠️ Aucun asset manager opérant dans le pays du client, opérant la stratégie adéquate, et/ou ayant le niveau de séniorité adapté au client proposé n'a été trouvé.")
        choice = input("Voulez-vous que le fonds recrute un asset manager adéquat (Oui) ou préférez-vous annuler l'enregistrement du client (Non) 📌 Entrez 'Oui' ou 'Non' : ")
        
        if choice == "Oui":
            assigned_manager = create_manager(client_data, db)
            
        
        elif choice == "Non":
            client_data = None
            print("❌ Enregistrement du client annulé.")
            sortie=True
        
        else:
            print("❌ Choix invalide ! Par défaut, on trouve un Asset Manager adapté.")
            assigned_manager = create_manager(client_data, db)

        
        if not sortie:
            print(f"✅ Un Manager a le profil correspondant: {assigned_manager['name']} ")
            choice = input(" Devons-nous le recruter? 📌 Entrez 'Oui' ou 'Non' : ")

            if choice == "Oui":
                print(assigned_manager)
                manager=AssetManager(**assigned_manager)
                client_data["manager_id"]=manager.save(db)
                print(f"✅ Manager {manager.name} recruté avec succès.")
            
            else:
                print("❌ Enregistrement du client annulé.")
                sortie=True
        

    ### Création du portefeuille à faire
    portfolio_data=create_portfolio(assigned_manager, client_data, db)  
    print("Portefeuille créé avec succès.")    

        
    if not sortie:

        ### Création du portefeuille à faire
        portfolio=Portfolio(**portfolio_data)

        #intégrer ici la vérification de l'existance de tous les assets dans la db
        #tickers_not_in_db = portfolio.check_assets(portfolio_data["products"], db)

        #if tickers_not_in_db:
            #print("❌ Les produits suivants ne sont pas dans la base de données : ", tickers_not_in_db)
            #téléchargement depuyis yahoo finance avec une fonction de data_collector
            #for ticker not in tickers_not_in_db:
                #asset=download_asset(ticker)
                #product=Product(**asset)
                #product.save(db)




        client_data["portfolio_id"]=portfolio.save(db) # à modifier
        print("Portefeuille créé avec succès.")

        print(client_data["name"], "est à présent un client de 'Data Management Project'.")
        
        client = Client(**client_data)
        client.save(db)
        
    db.close


except sqlite3.Error as e:
    print(f"Erreur SQLite : {e}")


/Users/guilhembarroyer/anaconda3/lib/python3.11/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


{'name': 'Christopher Taylor', 'age': 42, 'country': 'Mexico', 'email': 'christopher.taylor@yahoo.com', 'risk_profile': 'Low Risk', 'registration_date': '2022-10-10', 'investment_amount': 681396}
⚠️ Aucun asset manager opérant dans le pays du client, opérant la stratégie adéquate, et/ou ayant le niveau de séniorité adapté au client proposé n'a été trouvé.
✅ Un Manager a le profil correspondant: Teresa Smith 
{'name': 'Teresa Smith', 'age': 50, 'country': 'Mexico', 'email': 'teresa.smith@gmail.com', 'seniority': 'Senior', 'investment_sector': 'ms_utilities', 'strategies': ['Low Risk', 'High Risk'], 'clients_id': [1], 'portfolios_id': [1]}
✅ Manager Teresa Smith recruté avec succès.
Portefeuille créé avec succès.


TypeError: Portfolio.__init__() missing 1 required positional argument: 'id'